In [6]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# 我们先进行特征工程
## 该数据已经是经过标准化之后的了
data = pd.read_csv("diabetes.csv")
data.info()

# # 读取原始数据，并划分训练集和测试集
# raw_data = np.loadtxt('diabetes.csv', delimiter=',', dtype=np.float32)
# X = raw_data[:, :-1]
# y = raw_data[:, [-1]]
# Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,y,test_size=0.25)     #8:2比较好
# Xtest = torch.from_numpy(Xtest)  # array转torch，才能送到pytorch里面
# Ytest = torch.from_numpy(Ytest)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 758 entries, 0 to 757
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   -0.294118   758 non-null    float64
 1   0.487437    758 non-null    float64
 2   0.180328    758 non-null    float64
 3   -0.292929   758 non-null    float64
 4   0           758 non-null    float64
 5   0.00149028  758 non-null    float64
 6   -0.53117    758 non-null    float64
 7   -0.0333333  758 non-null    float64
 8   0.1         758 non-null    int64  
dtypes: float64(8), int64(1)
memory usage: 53.4 KB


In [8]:
data

-0.294118  0.487437  0.180328  ...  -0.53117  -0.0333333  0.1
0    -0.882353 -0.145729  0.081967  ... -0.766866   -0.666667    1
1    -0.058824  0.839196  0.049180  ... -0.492741   -0.633333    0
2    -0.882353 -0.105528  0.081967  ... -0.923997    0.000000    1
3     0.000000  0.376884 -0.344262  ...  0.887276   -0.600000    0
4    -0.411765  0.165829  0.213115  ... -0.894962   -0.700000    1
..         ...       ...       ...  ...       ...         ...  ...
753   0.176471  0.015075  0.245902  ... -0.920581    0.400000    1
754  -0.764706  0.226131  0.147541  ... -0.776260   -0.800000    1
755  -0.411765  0.216080  0.180328  ... -0.857387   -0.700000    1
756  -0.882353  0.266332 -0.016393  ... -0.768574   -0.133333    0
757  -0.882353 -0.065327  0.147541  ... -0.797609   -0.933333    1

[758 rows x 9 columns]

In [9]:
data.corr()

-0.294118  0.487437  0.180328  ...  -0.53117  -0.0333333       0.1
-0.294118    1.000000  0.143342  0.224508  ...  0.020092    0.286351 -0.218156
0.487437     0.143342  1.000000  0.218257  ...  0.130264    0.172119 -0.487703
0.180328     0.224508  0.218257  1.000000  ...  0.000987    0.229498 -0.151862
-0.292929    0.212215  0.168021  0.171751  ... -0.032794    0.140961 -0.195241
0            0.186578  0.202998  0.126856  ... -0.078075    0.218910 -0.157560
0.00149028   0.166938  0.231148  0.271711  ...  0.148917   -0.068091 -0.312509
-0.53117     0.020092  0.130264  0.000987  ...  1.000000    0.010858 -0.162625
-0.0333333   0.286351  0.172119  0.229498  ...  0.010858    1.000000 -0.110625
0.1         -0.218156 -0.487703 -0.151862  ... -0.162625   -0.110625  1.000000

[9 rows x 9 columns]

In [10]:
# 特征相关性分析可视化
# Correlation = pd.DataFrame(combined_train_test[
#  ['Embarked', 'Sex', 'Title', 'Name_length', 'Family_Size', 'Family_Size_Category','Fare', 'Fare_bin_id', 'Pclass', 
#   'Pclass_Fare_Category', 'Age', 'Ticket_Letter', 'Cabin']])

# colormap = plt.cm.viridis
# plt.figure(figsize=(14,12))
# plt.title('Pearson Correlation of Features', y=1.05, size=15)
# sns.heatmap(Correlation.astype(float).corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)



In [12]:
## 如果我们不做特征筛选，直接进网络
import numpy as np
import torch
import matplotlib.pyplot as plt
 
# prepare dataset
xy = np.loadtxt('diabetes.csv', delimiter=',', dtype=np.float32)
print(type(xy))
print(type(data.values))
## 可以看出我们用pandas和用loadtxt读取数据集本质上是等价的


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [13]:
print(xy)

[[-0.294118   0.487437   0.180328  ... -0.53117   -0.0333333  0.       ]
 [-0.882353  -0.145729   0.0819672 ... -0.766866  -0.666667   1.       ]
 [-0.0588235  0.839196   0.0491803 ... -0.492741  -0.633333   0.       ]
 ...
 [-0.411765   0.21608    0.180328  ... -0.857387  -0.7        1.       ]
 [-0.882353   0.266332  -0.0163934 ... -0.768574  -0.133333   0.       ]
 [-0.882353  -0.0653266  0.147541  ... -0.797609  -0.933333   1.       ]]


In [14]:
print(data.values)

[[-0.882353  -0.145729   0.0819672 ... -0.766866  -0.666667   1.       ]
 [-0.0588235  0.839196   0.0491803 ... -0.492741  -0.633333   0.       ]
 [-0.882353  -0.105528   0.0819672 ... -0.923997   0.         1.       ]
 ...
 [-0.411765   0.21608    0.180328  ... -0.857387  -0.7        1.       ]
 [-0.882353   0.266332  -0.0163934 ... -0.768574  -0.133333   0.       ]
 [-0.882353  -0.0653266  0.147541  ... -0.797609  -0.933333   1.       ]]


In [15]:
# 但是pandas会把第一行默认当作标签，就少了数据了
